In [1]:
import pandas as pd
import datetime
import numpy as np
import os
import re
import operator

## Отчет по продажам LFL

Необходимо обновить отчет за неделю - недели, последняя актуальная версия должна лежать здесь

\\tsclient\X\УМиР\2021\аналитика\отчет для руководства

Название "ОТЧЁТ ПО ПРОДАЖАМ LFL vR"

На текуйщи момент для того чтобы сделать отчет необходимо производить ручные выгрузки из Маркетинг --> Отчет по продажам

Делаем по сумме чеков, доходу, кол-ву чеков.

Интервалы:

до 100

100-300

300-500

500-1000

1000-2000

2000-2950

2950-3000

3000+


Т.е. понадобятся следующие выгрузки

100-300

300-500

500-1000

1000-2000

2000-2950

2950-3000

Файлы называем также и кладем в соответствующую папку:

Доход

Оборот

Чеки



Все файлы и папки необходимо сохранить в одну папку. Задать эту папку в FOLDER. Запустить и получим файл для обновления отчета.

In [2]:
FOLDER = r'D:\FTP-DATA\Analytics\LFL-query\23052021'

os.chdir(FOLDER)

## Определим папки из которых будем брать соотвествующие показатели

In [3]:
folders = [element for element in os.listdir() if os.path.isdir(element)]

## Создадим единый файл для показателей

In [4]:
indicators = pd.ExcelWriter('Показатели.xlsx')

### пройдемся по всем папкам и файлам

In [5]:
amount = pd.DataFrame()

intervals = {'100-300': {'Крупные': 'Удалить', 'Средние': '100-300', 'Мелкие': 'до 100', 'Итого оборот:': 'Удалить'},
            '300-500': {'Крупные': 'Удалить', 'Средние': '300-500', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'},
            '500-1000': {'Крупные': 'Удалить', 'Средние': '500-1000', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'},
            '1000-2000': {'Крупные': 'Удалить', 'Средние': '1000-2000', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'},
            '2000-2950': {'Крупные': 'Удалить', 'Средние': '2000-2950', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'},
            '2950-3000': {'Крупные': '3000+', 'Средние': '2950-3000', 'Мелкие': 'Удалить', 'Итого оборот:': 'Удалить'}}

def sort_func(arr):
    dct = {'до 100': 1,
'100-300': 2,
'300-500': 3,
'500-1000': 4,
'1000-2000': 5,
'2000-2950': 6,
'2950-3000': 7,
'3000+': 8}
    
    filials = dict(zip(sorted(list(set(temp['Отделение'].to_list()))), range(1, len(sorted(list(set(temp['Отделение'].to_list())))) + 1)))

    if 'Головное отделение' in arr.to_list():
        return [filials[i] for i in arr]
    else:
        return [dct[i] for i in arr]

sheets = {}

for folder in folders:
    temp = pd.DataFrame()
    for file in os.listdir(folder):
        if '$' not in file:
            temp_df = pd.read_excel(os.path.join(os.getcwd(), folder, file), skiprows=2)
            skip_footer_rows = temp_df.shape[0] - temp_df[temp_df.isna().all(axis=1)].iloc[0,:].name
            temp_df = pd.read_excel(os.path.join(os.getcwd(), folder, file), skiprows=2, skipfooter=skip_footer_rows)
            temp_df = temp_df[[col for col in temp_df.columns if col not in ('Итого:', 'Структура')]]
            temp_df = temp_df.fillna(method='ffill', axis=0)
            temp_df = temp_df.rename(columns={'Unnamed: 0': 'Отделение', 'Клиенты:': 'Интервал'})
            temp_df['Интервал'] = temp_df['Интервал'].map(intervals[file[:file.find('.')]])
            temp_df = temp_df[temp_df['Интервал'] != 'Удалить']
            temp = temp.append(temp_df)
    temp = temp.sort_values(by=['Отделение', 'Интервал'], key=sort_func)        
    sheets.setdefault(folder, temp)

for key in sheets.keys():
    sheets[key].to_excel(indicators, sheet_name=key, index=False)
                
       

## Сохраним все в файле Excel

In [6]:
indicators.save()
indicators.close()

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
